<a href="https://colab.research.google.com/github/marcogemaque/Hell-de-Janeiro/blob/main/Hell_de_Janeiro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Welcome to Hell-de-Janeiro 0.1.0**

This is a small app built in order to quantify the amount of shootings that happen in the marvelous city. The real focus in here is to "unormalize" the crazy amount of violence we have in Rio, by showing a map of the city and putting into perspective what do we know and also how does the media portray shootings depending on where do they happen.

## **Get Data from API**

In [ ]:
api_key = 'VYxaw4CVKpdtFGXgIzic5GHIl'
api_secret_key = 'Udiqt3beUN6kvKlrMT192Tq8aFNKhztJVOCyPCV5s5rLaEPWKM'
bearer = 'AAAAAAAAAAAAAAAAAAAAAO%2FwLwEAAAAA6YSq3ILZMkqiIYW7yOPrJqWPFys%3Dah7LH54oB6HD1KtAHnuc9YHArxlEGtFih2BrjZq4QMVMWDH2rI'
access_token = '1352216164953452546-HtITyrWZXE7WgibK5VQrAwB0yioCnR'
access_token_secret = 'GBQRAncELZAXxYLjN5AF9usNPGkMhUHKI5ZpeVNetS7Ps'

In [ ]:
!pip install tweepy

In [6]:
import requests
import json
import tweepy
import pandas as pd
import re
import time

In [ ]:
def connect_to_twitter_OAuth():
    auth = tweepy.OAuthHandler(api_key, api_secret_key)
    auth.set_access_token(access_token, access_token_secret)

    api = tweepy.API(auth)
    return api

def transform_tweets(tweet_obj):
    #Transform tweet object texts to lists
    list_tweets = [] #Empty list to store tweets
    for tweet in tweet_obj:
        tweet_date = tweet.created_at
        tweet_text = tweet.text
        tweet_id = tweet.id
        list_tweets.append({'date':tweet_date,
                            'content':tweet_text,
                            'id':tweet_id})
        
    #Create dataframe
    df = pd.DataFrame(list_tweets, columns=['date','content','id'])
    return df

#Create API object
api = connect_to_twitter_OAuth()

#Create function to retrieve more tweets
def twitter_scraper():
    api = connect_to_twitter_OAuth()
    ott = api.user_timeline('RJ_OTT', count=200)
    df_inside = transform_tweets(ott)
    result = df_inside
    for i in range(59):
        try:
            time.sleep(1) #sleep for 1 second
            last_id = result['id'].values[-1]
            ott = api.user_timeline('RJ_OTT', count=200, max_id=str(last_id))
            df_inner = transform_tweets(ott)
            list_dfs = [result, df_inner]
            result = pd.concat(list_dfs)
            result = result.sort_values(by=['id'], ascending=False)
        except:
            pass
    return result

df_tweets = twitter_scraper()
df = df_tweets.copy() #create a copy so we can always change the outlet dataframe

In [ ]:
df.tail()

,date,content,id
0,2019-04-02 09:35:02,OTT-RJ Informa:\n\n02/04 - 06:25h\n\nTiros na ...,1113011950844329985
0,2019-04-02 09:35:02,OTT-RJ Informa:\n\n02/04 - 06:25h\n\nTiros na ...,1113011950844329985
0,2019-04-02 09:35:02,OTT-RJ Informa:\n\n02/04 - 06:25h\n\nTiros na ...,1113011950844329985
0,2019-04-02 09:35:02,OTT-RJ Informa:\n\n02/04 - 06:25h\n\nTiros na ...,1113011950844329985
0,2019-04-02 09:35:02,OTT-RJ Informa:\n\n02/04 - 06:25h\n\nTiros na ...,1113011950844329985


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3262 entries, 0 to 0
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     3262 non-null   datetime64[ns]
 1   content  3262 non-null   object        
 2   id       3262 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 101.9+ KB


## **Wrangle the database**

We have some ways in which the tweets were written. The main captured standards were: <br>
1) Local: xxx, Rio de Janeiro <br>
2) Local: xxx <br>
3) Tiros ouvidos na/em xxx <br>
4) Tiros na/em xxx <br>
5) Disparos ouvidos em/no xxx <br>
A major way to split would be to catch the first occurence of a comma (","), since this is a common character splitting. We need regex for all other standards as well. Perhaps a common function would be better to use which comprises of all the possible methods instead of writing multiple lines of regex. But we'll see.

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
df = pd.read_csv('/content/drive/My Drive/Hell-de-Janeiro App/output.csv')

In [9]:
#Drop columns
df = df.drop(['Unnamed: 0'], axis=1)

In [10]:
df.head()

,date,content,id
0,2021-01-24 13:08:29,Tiroteio - 24/01/2021 às 10:06\nLocal: Pilares...,1353328823266398216
1,2021-01-24 03:27:35,Disparos ouvidos - 24/01/2021 às 00:20h\nLocal...,1353182632897622017
2,2021-01-24 03:22:18,Disparos ouvidos - 24/01/2021 às 00:00h\nLocal...,1353181303441682438
3,2021-01-24 01:13:09,OTT-RJ INFORMA:\n\n23/01 - 21:43h\nTiros emBel...,1353148801989672961
4,2021-01-23 06:51:57,Tiroteio - 23/01/2021 às 03:36h\nLocal: Andara...,1352871674924957696


In [33]:
"""
So this is how things are going to be.
1) Create regex expression to filter dataframe
2) Get indexes of filtered entries by regex
3) On to the next regex expression, remove the indexes from the previous created regex
"""

#Standard Local: xxx, Rio de Janeiro or Local: xxx
df_out = df['content'].str.split(r"\s*Local:", expand=True)[1]
df_out = df_out.str.split(r"\s*Rio de Janeiro|\n", expand=True)[0]
df_out = df_out.str.replace(r"^\s", "")
df_out = pd.DataFrame(df_out)
df_out = df_out.dropna()
df_out.columns = ['Place']

list_index = df_out.index.values.tolist()

#Get the same dataframe, but this time remove the indexes you have retrieved
df_own = df.drop(list_index)

#Get the second regex
df_out2 = df_own['content'].str.split(r"\s*Tiros em|Tiros na", expand=True)[1]
df_out2 = df_out2.str.split(",", expand=True)[0]
df_out2 = df_out2.str.replace(r"^\s", "")
df_out2 = pd.DataFrame(df_out2)
df_out2 = df_out2.dropna()
df_out2.columns = ['Place']

#Get the same dataframe, but this time remove the indexes you have retrieved before
list_index.extend(df_out2.index.values.tolist())
df_own = df.drop(list_index)

#Get the third regex
df_out3 = df_own['content'].str.split(r"\s*no |em |na ", expand=True)[1]
df_out3 = df_out3.str.split(",", expand=True)[0]
df_out3 = df_out3.str.replace(r"^\s", "")
df_out3 = pd.DataFrame(df_out3)
df_out3 = df_out3.dropna()
df_out3.columns = ['Place']
df_out3.head()

#Get the same dataframe, but this time remove the indexes you have retrieved before
list_index.extend(df_out3.index.values.tolist())
df_own = df.drop(list_index)

In [34]:
df_work = pd.DataFrame()
df_work['Place'] = df_out['Place']
df_work = pd.concat([df_work, df_out2, df_out3])
df_work['date'] = df['date'].iloc[df_work.index.values.tolist()]
df_work = df_work.reset_index(drop=True)

In [35]:
df_work['Place'] = df_work['Place'].str.replace(r"^\s", "") #replaces blank space ahead
df_work['Place'] = df_work['Place'].str.replace(r"\s$", "") #replaces blank space at the end or dot
df_work['Place'] = df_work['Place'].apply(lambda x: x.split("(")[0] if "(" in x else x) #removes everything after "("
df_work['Place'] = df_work['Place'].apply(lambda x: x.split("-")[0] if "-" in x else x) #removes everything after "-"
df_work['Place'] = df_work['Place'].apply(lambda x: x.split("/")[0] if "/" in x else x) #removes everything after "/"
df_work['Place'] = df_work['Place'].apply(lambda x: x.split(".")[0] if "." in x else x) #removes everything after "."

In [36]:
df_work['Place'].unique()

array(['Pilares', 'Rio Comprido', 'Piedade', 'Andaraí', 'Praça Seca',
       'Cascadura', 'Belford Roxo', 'Brás de Pina', 'Maré',
       'Duque de Caxias', 'Quintino Bocaiuva', 'Senador Camará',
       'Catumbi', 'Complexo do Alemão', 'Madureira', 'Costa Barros',
       'Vila Kennedy', 'Leme', 'Angra dos Reis', 'São Gonçalo', 'Japeri',
       'Cosme Velho', 'Jardim América', 'Caju', 'Mangueira', 'Magé',
       'Centro', 'Cosme Velho ', 'Glória', 'Piedade ', 'Belford Roxo ',
       'Costa Barros ', 'Madureira ', 'Complexo do Alemão ', 'Cavalcanti',
       'Quintino', 'Jacarezinho', 'São Gonçalo ', 'Engenho da Rainha',
       'Complexo da Maré', 'Penha ', 'Mesquita ', 'Morro do Estado',
       'Praça da Bandeira', 'Vila Isabel', 'Complexo da Penha',
       'Santa Teresa ', 'Cordovil', 'Manguinhos', 'Anchieta',
       'Cidade de Deus', 'Penha', 'Bonsucesso', 'Grajaú',
       'Belford Roxo  ', 'Engenheiro Pedreira ', 'Senador Camará  ',
       'Vila Kennedy ', 'Caxias', 'Copacabana', 'Mesq

We have filtered the dataframe as best as we can, now we need to clean out the wrong values.

In [ ]:
list_vals = df_work['Place'].unique().tolist()
list_vals

['Pilares',
 'Rio Comprido',
 'Piedade',
 'Andaraí',
 'Praça Seca',
 'Cascadura',
 ' Belford Roxo',
 'Brás de Pina',
 'Maré',
 ' Duque de Caxias',
 'Quintino Bocaiuva',
 'Senador Camará',
 'Catumbi',
 'Complexo do Alemão',
 'Madureira',
 'Costa Barros',
 'Vila Kennedy',
 'Leme',
 ' Angra dos Reis',
 ' São Gonçalo',
 ' Japeri',
 'Cosme Velho',
 'Jardim América',
 'Caju',
 'Mangueira',
 ' Magé',
 'Centro',
 ' Cosme Velho (Cerro Corá)',
 'Glória/Catete ( Santo Amaro)',
 'Piedade (morro do Urubu)',
 'Belford Roxo (Santa Teresa)',
 'Costa Barros (Caminho do Padre)',
 'Madureira (Patolinha)',
 'Complexo do Alemão (Parte da Mata)',
 'Cavalcanti',
 'Quintino',
 'Jacarezinho',
 'São Gonçalo / Colubandê',
 'Engenho da Rainha',
 'Complexo da Maré',
 'Penha (Vila Cruzeiro)',
 'Mesquita (Coréia)',
 'Rio Comprido ',
 'Morro do Estado ',
 'Praça da Bandeira ',
 'Vila Isabel',
 'Complexo da Penha ',
 'Santa Teresa - Belford Roxo ',
 'Cordovil ',
 'Catumbi ',
 'Manguinhos ',
 'Anchieta',
 'Cidade de De

## **Frustrated Dev Diary**

**Entry 21-01-2021** <br>
Another day, another project. Created Twitter's development account and officially began the project. Shoutout to @saramalvar for supporting it and helping on it. <br>

**Entry 22-01-2021:** <br>
Tweets have been posted on a dataframe. Df's limit is of 20 tweets by the method
we imported, so we'll have to think a way of expanding it, but we'll create the
regex for the 20 just as a start. We have tweet date (to take the year and month) and text so we can create the real dataframe with divisions by columns. <br>
The idea is: if the dataframe doesn't have place description then we disregard the input. <br>

**Entry 25-01-2021:** <br>
I believe we collected almost all useful tweets. We used 3 sets of regex and ran on our database of tweets collected from the API. Out of the 3200 tweets we used 3000. The remanining ones (tails and heads of df checked for this) seemed to be mentions asking where did the shootings happened and other similar. Branding as well.